<a href="https://colab.research.google.com/github/alirezaght/tesseract-train-font/blob/main/tesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting google drive to have access to data

In [ ]:
from google.colab import drive
drive.mount(‘/content/drive’)

# Install tesseract

First let's install tesseract + all the dependency required

In [1]:
!apt install tesseract-ocr libtesseract-dev bc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  bc libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 37 not upgraded.
Need to get 7,636 kB of archives.
After this operation, 29.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 bc amd64 1.07.1-2 [86.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libleptonica-dev amd64 1.75.3-3 [1,308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtesseract-dev amd64 4.00~git2288-10f4998a-2 [1,447 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesser

# Clone tesstrain

In [2]:
!git clone https://github.com/tesseract-ocr/tesstrain

Cloning into 'tesstrain'...
remote: Enumerating objects: 819, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 819 (delta 21), reused 18 (delta 9), pack-reused 775
Receiving objects: 100% (819/819), 13.37 MiB | 25.22 MiB/s, done.
Resolving deltas: 100% (463/463), done.


# Prepare data

In [4]:
!cp /content/drive/MyDrive/dataset.rar /content/
!unrar x /content/dataset.rar
!mkdir tesstrain/data
!mv bnk-ground-truth tesstrain/data/bnk-ground-truth
%cd /content/tesstrain

Streaming output truncated to the last 5000 lines.
Extracting  bnk-ground-truth/12d918c7-c454-4e2a-be8a-9576b5f0fe6d.gt.txt       7%  OK 
Extracting  bnk-ground-truth/12d918c7-c454-4e2a-be8a-9576b5f0fe6d.tif       7%  OK 
Extracting  bnk-ground-truth/12e5d1aa-1b98-4fa8-8067-88ca7830c941.gt.txt       7%  OK 
Extracting  bnk-ground-truth/12e5d1aa-1b98-4fa8-8067-88ca7830c941.tif       7%  OK 
Extracting  bnk-ground-truth/12f18be4-1a1a-4e72-90c9-d00f060a3b8e.gt.txt       7%  OK 
Extracting  bnk-ground-truth/12f18be4-1a1a-4e72-90c9-d00f060a3b8e.tif       7%  OK 
Extracting  bnk-ground-truth/1337a7b5-36a4-4db6-9425-2983630f66fe.gt.txt       7%  OK 
Extracting  bnk-ground-truth/1337a7b5-36a4-4db6-9425-2983630f66fe.tif       7%  OK 
Extracting  bnk-ground-truth/134e1787-6f3c-4853-bd8e-354a6ae37328.gt.txt       7%  OK 
Extracting  bnk-ground-truth/134e1787-6f3c-4853-bd8e-354a6ae37328.tif       7%  OK 
Extr

# Train

In [ ]:
!make training MODEL_NAME=bnk

find -L data/bnk-ground-truth -name '*.gt.txt' | xargs paste -s > "data/bnk/all-gt"
unicharset_extractor --output_unicharset "data/bnk/unicharset" --norm_mode 2 "data/bnk/all-gt"
Bad box coordinates in boxfile string! 6407782187936793
Extracting unicharset from plain text file data/bnk/all-gt
Wrote unicharset file data/bnk/unicharset
PYTHONIOENCODING=utf-8 python3 generate_line_box.py -i "data/bnk-ground-truth/1f97ba7b-a697-4b76-8b4b-2305bc0e71ec.tif" -t "data/bnk-ground-truth/1f97ba7b-a697-4b76-8b4b-2305bc0e71ec.gt.txt" > "data/bnk-ground-truth/1f97ba7b-a697-4b76-8b4b-2305bc0e71ec.box"
+ tesseract data/bnk-ground-truth/1f97ba7b-a697-4b76-8b4b-2305bc0e71ec.tif data/bnk-ground-truth/1f97ba7b-a697-4b76-8b4b-2305bc0e71ec --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.0.0-beta.1 with Leptonica
Page 1
Warning. Invalid resolution 1 dpi. Using 70 instead.
PYTHONIOENCODING=utf-8 python3 generate_line_box.py -i "data/bnk-ground-truth/1524de56-5e4b-4fa1-811d-3881d764133a.tif" -t "data/b

In [ ]:
!make traineddata MODEL_NAME=bnk

mkdir data/bnk/tessdata_best
mkdir data/bnk/tessdata_fast
lstmtraining \
          --stop_training \
          --continue_from data/bnk/checkpoints/bnk31.234_344.checkpoint \
          --traineddata data/bnk/bnk.traineddata \
          --model_output data/bnk/tessdata_best/bnk31.234_344.traineddata
Loaded file data/bnk/checkpoints/bnk31.234_344.checkpoint, unpacking...
lstmtraining \
          --stop_training \
          --continue_from data/bnk/checkpoints/bnk15.231_372.checkpoint \
          --traineddata data/bnk/bnk.traineddata \
          --model_output data/bnk/tessdata_best/bnk15.231_372.traineddata
Loaded file data/bnk/checkpoints/bnk15.231_372.checkpoint, unpacking...
lstmtraining \
          --stop_training \
          --continue_from data/bnk/checkpoints/bnk0_422.checkpoint \
          --traineddata data/bnk/bnk.traineddata \
          --model_output data/bnk/tessdata_best/bnk0_422.traineddata
Loaded file data/bnk/checkpoints/bnk0_422.checkpoint, unpacking...
lstmtraining \
